In [102]:
train_path = "./datasets/WASSA23_essay_level_internal_train_preproc.tsv"
val_path = "./datasets/WASSA23_essay_level_internal_val_preproc.tsv"
dev_path = "./datasets/WASSA23_essay_level_dev_preproc.tsv"
test_path = "./datasets/WASSA23_essay_level_test_preproc.tsv"

In [103]:
import pandas as pd

train_df = pd.read_csv(train_path, sep='\t')
val_df = pd.read_csv(val_path, sep='\t')
dev_df = pd.read_csv(dev_path, sep='\t')
test_df = pd.read_csv (test_path, sep='\t')

In [104]:
train_df.columns

Index(['conversation_id', 'article_id', 'essay', 'empathy', 'distress',
       'speaker_id', 'gender', 'education', 'race', 'age', 'income',
       'personality_conscientiousness', 'personality_openess',
       'personality_extraversion', 'personality_agreeableness',
       'personality_stability', 'iri_perspective_taking',
       'iri_personal_distress', 'iri_fantasy', 'iri_empathatic_concern',
       'speaker_number', 'split', 'essay_id', 'emotion', 'anger_count',
       'disgust_count', 'fear_count', 'joy_count', 'sadness_count',
       'surprise_count', 'hope_count', 'empathy_count', 'distress_count'],
      dtype='object')

In [105]:
from nrclex import NRCLex
import numpy as np

In [109]:
def add_prompt(data):
    data["prompt"] = ""
    for idx, row in data.iterrows():
        if row['gender'] == 1: gender = "male"
        else: gender = "female"

        if row['education'] == 1: edu = "less than a high school diploma"
        elif row['education'] == 2: edu = "high School diploma"
        elif row['education'] == 3: edu = "technical/vocational School"
        elif row['education'] == 4: edu = "some college but no degree"
        elif row['education'] == 5: edu = "two year associate degree"
        elif row['education'] == 6: edu = "four year bachelor's degree"
        else: edu = "postgradute or professional degree"

        if row['race'] == 1: ethnicity = "white"
        elif row['race'] == 2: ethnicity = "hispanic or latino"
        elif row['race'] == 3: ethnicity = "black or african american"
        elif row['race'] == 4: ethnicity = "native american or american indian"
        elif row['race'] == 5: ethnicity = "asian/pacific islander"
        else: ethnicity = ""

        text_prompt_bio = "An essay written by a {} years old {} {}, with {}, with an income of {}$.".format(row["age"], ethnicity, gender,
                    edu, row["income"]) 
        
        if row["empathy"] < 3: emp = "The essay expresses low empathy"
        elif row["empathy"] < 5: emp = "The essay not expresses empathy"
        else: emp = "The essay expresses high empathy"
        if row["distress"] < 3: dis = "low"
        elif row["distress"] < 5: dis = "medium"
        else: dis = "high"
        text_prompt_emp = " {} and {} distress level.".format(emp,  dis)

        emotions = NRCLex(row["essay"]).top_emotions
        if (sum(np.array([emo[1] for emo in emotions])))== 0:
           emotions = {'neutral': 0}
        n_emo = len(emotions)
        string = ""
        for i, emo in enumerate(emotions):
            string += emo[0]
            if i < n_emo-1:
                string += ", "
        text_prompt_emo = " Top emotions expressed by the writer are: {}.".format(string)

        text_prompt = row["essay"] + '"' + text_prompt_bio + text_prompt_emp + text_prompt_emo + '"'
        data["prompt"][idx] = text_prompt



In [110]:
add_prompt(train_df)
add_prompt(val_df)
add_prompt(dev_df)
#add_prompt(test_df)

C:\Users\giuli\AppData\Local\Temp\ipykernel_16508\1296604898.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["prompt"][idx] = text_prompt
C:\Users\giuli\AppData\Local\Temp\ipykernel_16508\1296604898.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["prompt"][idx] = text_prompt
C:\Users\giuli\AppData\Local\Temp\ipykernel_16508\1296604898.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["prompt"][idx] = text_pr

In [111]:
train_df

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count,empathy_count,distress_count,prompt
0,193,313,It's so sad the turning event of Immigrants st...,6.333333,6.125,36,1,7,1,46,...,6.896552,3.448276,10.344828,5.172414,15.517241,3.448276,0,8.220577,7.137115,It's so sad the turning event of Immigrants st...
1,340,223,I feel bad for the mother and her two daughter...,6.500000,2.875,30,1,6,3,37,...,9.230769,7.692308,18.461538,15.384615,24.615385,12.307692,0,6.851257,6.467713,I feel bad for the mother and her two daughter...
2,392,270,I feel so bad for the victims and the victims ...,6.000000,6.000,53,2,3,1,27,...,8.943089,8.130081,10.569106,9.756098,9.756098,8.130081,0,3.762797,3.606506,I feel so bad for the victims and the victims ...
3,304,347,"One of the biggest fears of a parent, is that ...",6.666667,6.375,30,1,6,3,37,...,2.597403,1.298701,3.896104,19.480519,10.389610,6.493506,0,6.110432,5.607771,"One of the biggest fears of a parent, is that ..."
4,263,34,I feel sorry for the people living in that reg...,6.666667,6.625,30,1,6,3,37,...,4.411765,4.411765,10.294118,14.705882,7.352941,10.294118,0,6.668359,6.151027,I feel sorry for the people living in that reg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,266,47,Reading about the situation in Syria is always...,4.000000,3.125,25,1,4,1,29,...,3.125000,0.000000,7.812500,3.125000,3.125000,0.000000,0,7.022593,6.635832,Reading about the situation in Syria is always...
613,319,39,I just read about an unfortunate wolf that had...,5.833333,4.750,25,1,4,1,29,...,10.169492,8.474576,8.474576,1.694915,11.864407,5.084746,0,7.559620,7.106100,I just read about an unfortunate wolf that had...
614,419,301,I had to read this article over twice because ...,6.166667,6.000,19,1,6,2,32,...,4.285714,2.857143,2.857143,7.142857,5.714286,5.714286,0,6.532548,6.238817,I had to read this article over twice because ...
615,479,53,I just read an article about the amount of foo...,1.000000,2.625,25,1,4,1,29,...,2.898551,2.898551,1.449275,8.695652,4.347826,0.000000,0,5.986728,5.726251,I just read an article about the amount of foo...


In [114]:
train_df

,conversation_id,article_id,essay,empathy,distress,speaker_id,gender,education,race,age,...,anger_count,disgust_count,fear_count,joy_count,sadness_count,surprise_count,hope_count,empathy_count,distress_count,prompt
0,193,313,It's so sad the turning event of Immigrants st...,6.333333,6.125,36,1,7,1,46,...,6.896552,3.448276,10.344828,5.172414,15.517241,3.448276,0,8.220577,7.137115,It's so sad the turning event of Immigrants st...
1,340,223,I feel bad for the mother and her two daughter...,6.500000,2.875,30,1,6,3,37,...,9.230769,7.692308,18.461538,15.384615,24.615385,12.307692,0,6.851257,6.467713,I feel bad for the mother and her two daughter...
2,392,270,I feel so bad for the victims and the victims ...,6.000000,6.000,53,2,3,1,27,...,8.943089,8.130081,10.569106,9.756098,9.756098,8.130081,0,3.762797,3.606506,I feel so bad for the victims and the victims ...
3,304,347,"One of the biggest fears of a parent, is that ...",6.666667,6.375,30,1,6,3,37,...,2.597403,1.298701,3.896104,19.480519,10.389610,6.493506,0,6.110432,5.607771,"One of the biggest fears of a parent, is that ..."
4,263,34,I feel sorry for the people living in that reg...,6.666667,6.625,30,1,6,3,37,...,4.411765,4.411765,10.294118,14.705882,7.352941,10.294118,0,6.668359,6.151027,I feel sorry for the people living in that reg...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,266,47,Reading about the situation in Syria is always...,4.000000,3.125,25,1,4,1,29,...,3.125000,0.000000,7.812500,3.125000,3.125000,0.000000,0,7.022593,6.635832,Reading about the situation in Syria is always...
613,319,39,I just read about an unfortunate wolf that had...,5.833333,4.750,25,1,4,1,29,...,10.169492,8.474576,8.474576,1.694915,11.864407,5.084746,0,7.559620,7.106100,I just read about an unfortunate wolf that had...
614,419,301,I had to read this article over twice because ...,6.166667,6.000,19,1,6,2,32,...,4.285714,2.857143,2.857143,7.142857,5.714286,5.714286,0,6.532548,6.238817,I had to read this article over twice because ...
615,479,53,I just read an article about the amount of foo...,1.000000,2.625,25,1,4,1,29,...,2.898551,2.898551,1.449275,8.695652,4.347826,0.000000,0,5.986728,5.726251,I just read an article about the amount of foo...


In [113]:
train_df.to_csv(f"datasets/WASSA23_essay_level_internal_train_preproc.tsv", index=False, sep='\t')
val_df.to_csv(f"datasets/WASSA23_essay_level_internal_val_preproc.tsv", index=False, sep='\t')
dev_df.to_csv(f"datasets/WASSA23_essay_level_dev_preproc.tsv", index=False, sep='\t')
test_df.to_csv(f"datasets/WASSA23_essay_level_test_preproc.tsv", index=False, sep='\t')
    